# Script to integrate EggNOG annotation with KBase model construction

In [ ]:
! pip install biopython
! pip install cobra
! pip install gffpandas

In [26]:
#%% Import required libraries
import os
import re
import json
import requests
from datetime import datetime
import pandas as pd
from Bio import SeqIO
import gffpandas.gffpandas as gffpd


## Parse EggNOG file, gene call, fasta file, Model Seed and Transporter file to produce table of imputed reactions and transportable metabolites


Files needed (example for MIT1002)


*   EggNOG annotation output file: eggnog_output.emapper.annotations
*   Identifier for transporters (from https://www.tcdb.org/): TCDB_transporter_substrates.txt
*   Protein annotation file: MIT1002_anvio_prot_seqs.fa  
*   Gene call file, to import start/end locus of each gene: 2738541267_genecalls.gff
* SBML file generatted by KBase (used to associate with each gene how it was used by ModelSeed Draft): mit1002_rast_omegga.mdl.xml
* Output of CarveMe, to display CarveMe reactions and confidence score: carveme_model_reaction_scores.tsv

Script also calls ModelSee reactions and compounds. Information on formats is here:
https://github.com/ModelSEED/ModelSEEDDatabase/blob/master/Biochemistry/REACTIONS.md
https://github.com/ModelSEED/ModelSEEDDatabase/blob/master/Biochemistry/COMPOUNDS.md




### The following script translates the EggNOG annotation into a big data table, uses KEGG reaction IDs in EggNOG to make inferences of reactions (in ModelSeed format), and uses TCDB transporter identifiers to suggest transportable metabolites. All new info is added to the EggNOG table (called eggono_data)

In [33]:
#%% Function to parse the eggnog annotations file
def parse_eggnog(filename):
    """
    Parses the eggnog file into a dictionary keyed by the "query" field.
    Each record is stored as a dictionary of all columns.
    """
    eggnog_data = {}
    header = None
    with open(filename, 'r') as f:
        for line in f:
            line = line.rstrip("\n")
            if line.startswith("#"):
                if header is None and "query" in line.lower():
                    header = line.lstrip("#").split("\t")
                continue
            if not line.strip():
                continue
            if header is None:
                header = line.split("\t")
                continue
            fields = line.split("\t")
            record = dict(zip(header, fields))
            query_id = record["query"]
            eggnog_data[query_id] = record
    return eggnog_data

#%% Function to parse the protein FASTA file
def parse_fasta(filename):
    """
    Parses a FASTA file and returns a dictionary mapping gene IDs (from record.id) to sequences.
    """
    fasta_dict = {}
    for record in SeqIO.parse(filename, "fasta"):
        fasta_dict[record.id] = str(record.seq)
    return fasta_dict

#%% Function to parse a GFF file and extract gene locus (start-end) - this works for gff from ANVIO
def parse_gff(filename):
    """
    Parses a GFF file and returns a dictionary mapping gene IDs (from the "ID" attribute)
    to a dictionary containing locus information (here only start and end positions).
    """
    gff_dict = {}
    with open(filename, 'r') as f:
        for line in f:
            if line.startswith("#"):
                continue
            parts = line.strip().split("\t")
            if len(parts) < 9:
                continue
            seqid, source, feature_type, start, end, score, strand, phase, attributes = parts
            attr_dict = {}
            for attr in attributes.split(";"):
                if "=" in attr:
                    key, value = attr.split("=", 1)
                    attr_dict[key.strip()] = value.strip()
            if "ID" in attr_dict:
                gene_id = attr_dict["ID"]
                gff_dict[gene_id] = {"start": start, "end": end}
    return gff_dict

#%% Alternative gff parser that works for standard gff files from NCBI
# def parse_gff_v2(filename):
#     """
#     Parses a GFF file and returns a dictionary mapping gene IDs (from the "ID" attribute)
#     to a dictionary containing locus information (start and end positions).
#     """
#     gff_dict = {}

#     # Read GFF file using gffpandas
#     gff_df = gffpd.read_gff3(filename).df

#     # Filter only 'gene' features
#     gene_df = gff_df[gff_df["type"] == "gene"]

#     for _, row in gene_df.iterrows():
#         attributes = row["attributes"]
#         attr_dict = dict(item.split("=") for item in attributes.split(";") if "=" in item)

#         if "ID" in attr_dict:
#             gene_id = attr_dict["ID"]
#             gff_dict[gene_id] = {"start": int(row["start"]), "end": int(row["end"])}

#     return gff_dict

# import gffpandas.gffpandas as gffpd

def parse_gff_v2(filename):
    """
    Parses a GFF file and returns a dictionary mapping gene or CDS IDs
    (extracted from the 'ID' attribute without prefixes)
    to a dictionary containing locus information (start and end positions).
    """
    gff_dict = {}

    # Read GFF file using gffpandas
    gff_df = gffpd.read_gff3(filename).df

    # Filter only 'gene' and 'CDS' features
    filtered_df = gff_df[gff_df["type"].isin(["gene", "CDS"])]

    for _, row in filtered_df.iterrows():
        attributes = row["attributes"]
        attr_dict = dict(item.split("=") for item in attributes.split(";") if "=" in item)

        if "ID" in attr_dict:
            gene_id = attr_dict["ID"].split("-")[-1]  # Extract the ID without prefix
            gff_dict[gene_id] = {"start": int(row["start"]), "end": int(row["end"])}

    return gff_dict

# Example usage:
# genes = parse_gff_v2("example.gff")
# print(genes)


# Example usage:
# genes = parse_gff_v2("example.gff")
# print(genes)



#%% Build a cleaned mapping from KEGG reaction numbers to ModelSEED reaction entries.
def build_cleaned_kegg_to_modelseed_mapping(reactions_url):
    """
    Downloads the ModelSEED reactions JSON file, filters for reactions with status "OK",
    extracts the KEGG reaction ID from the aliases, and de-duplicates reactions (by equation).

    Returns a dictionary mapping KEGG reaction IDs (upper-case) to a list of reaction entries,
    each as a dict with keys "id" and "name".
    """
    response = requests.get(reactions_url)
    reactions_data = response.json()
    total_reactions = len(reactions_data)
    print(f"Downloaded {total_reactions} reactions.")

    # Filter reactions: keep only those with status "OK"
    ok_reactions = [rxn for rxn in reactions_data if rxn.get("status", "").strip().upper() == "OK"]
    print(f"Reactions with status 'OK': {len(ok_reactions)} (out of {total_reactions} downloaded)")

    # Extract KEGG Reaction ID from aliases.
    for rxn in ok_reactions:
        aliases = rxn.get("aliases")
        kegg_rxn = None
        if aliases and isinstance(aliases, list):
            for alias in aliases:
                alias_lower = alias.lower()
                if alias_lower.startswith("kegg:"):
                    kegg_rxn = alias.split(":", 1)[1].strip()
                    break
                elif alias_lower.startswith("kegg_"):
                    kegg_rxn = alias.split("_", 1)[1].strip()
                    break
        rxn["kegg_reaction"] = kegg_rxn

    # Deduplicate reactions by KEGG Reaction ID and equation.
    kegg_to_rxn = {}
    def get_rxn_number(rxn_id):
        m = re.search(r'\d+', rxn_id)
        return int(m.group(0)) if m else float('inf')

    for rxn in ok_reactions:
        kegg_id = rxn.get("kegg_reaction")
        if not kegg_id:
            continue
        kegg_id = kegg_id.strip().upper()
        if kegg_id not in kegg_to_rxn:
            kegg_to_rxn[kegg_id] = {}
        eq = rxn.get("equation", "").strip()
        if not eq:
            continue
        if eq not in kegg_to_rxn[kegg_id]:
            kegg_to_rxn[kegg_id][eq] = rxn
        else:
            existing_rxn = kegg_to_rxn[kegg_id][eq]
            existing_id = existing_rxn.get("id", "")
            new_id = rxn.get("id", "")
            if get_rxn_number(new_id) < get_rxn_number(existing_id):
                kegg_to_rxn[kegg_id][eq] = rxn

    # Build final mapping: for each KEGG reaction ID, collect reaction entries with "id" and "name".
    kegg_mapping = {}
    for kegg_id, eq_dict in kegg_to_rxn.items():
        reactions_list = []
        for eq, rxn in eq_dict.items():
            reactions_list.append({"id": rxn.get("id", ""), "name": rxn.get("name", "")})
        kegg_mapping[kegg_id] = reactions_list
    return kegg_mapping

#%% Parse transporter annotation file.
def parse_transporter(filename):
    """
    Parses the TCDB transporter substrates file.
    Each line is tab-delimited:
      - First column: transporter (TC) code.
      - Second column: one or more substrate entries separated by "|".
    Each substrate entry is in the format "CHEBI:xxxx;CommonName".

    Returns a dictionary mapping transporter codes to a list of metabolite dictionaries,
    each with keys "chebi" and "name".
    """
    transporter_dict = {}
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"):
                continue
            parts = line.split("\t")
            if len(parts) >= 2:
                tc_code = parts[0].strip()
                substrates_field = parts[1].strip()
                substrate_entries = substrates_field.split("|")
                metabolites = []
                for entry in substrate_entries:
                    subparts = entry.split(";")
                    if len(subparts) >= 2:
                        chebi_id = subparts[0].strip()  # e.g. "CHEBI:29035"
                        common_name = subparts[1].strip()
                    else:
                        chebi_id = ""
                        common_name = entry.strip()
                    metabolites.append({"chebi": chebi_id, "name": common_name})
                transporter_dict.setdefault(tc_code, []).extend(metabolites)
    return transporter_dict

#%% Main processing ===========================================================

# 1. Parse the eggnog file.
eggnog_file = "eggnog_output.emapper.annotations"
eggnog_data = parse_eggnog(eggnog_file)
print(f"Parsed eggnog file: {len(eggnog_data)} records found.")

# 2. Parse the protein FASTA file.
fasta_file = "protein.faa"
fasta_dict = parse_fasta(fasta_file)
print(f"Parsed FASTA file: {len(fasta_dict)} protein sequences found.")

# 3. Parse the GFF file.
gff_file = "genomic.gff"
#gff_dict = parse_gff(gff_file)
gff_dict = parse_gff_v2(gff_file)
print(f"Parsed GFF file: {len(gff_dict)} gene locus entries found.")

# 4. Associate protein sequences and locus information (start-end) with each eggnog gene record.
for query_id, record in eggnog_data.items():
    record["protein_seq"] = fasta_dict.get(query_id, "")
    locus = gff_dict.get(query_id, {})
    record["locus"] = f"{locus.get('start','')}-{locus.get('end','')}" if locus else ""

# 5. Download and parse ModelSEED compounds and reactions.
compounds_url = "https://raw.githubusercontent.com/ModelSEED/ModelSEEDDatabase/master/Biochemistry/compounds.json"
reactions_url = "https://raw.githubusercontent.com/ModelSEED/ModelSEEDDatabase/master/Biochemistry/reactions.json"

print("Downloading ModelSEED compounds...")
compounds_response = requests.get(compounds_url)
compounds_data = compounds_response.json()
print(f"Downloaded {len(compounds_data)} compounds.")

print("Cleaning and deduplicating ModelSEED reactions...")
kegg_mapping = build_cleaned_kegg_to_modelseed_mapping(reactions_url)
print(f"Built cleaned KEGG-to-ModelSEED mapping: {len(kegg_mapping)} KEGG reaction entries mapped.")

# 6. For each eggnog gene record with a KEGG reaction number, add the corresponding ModelSEED reaction entries.
#    Each entry will be a dict with keys "id" and "name".
for query_id, record in eggnog_data.items():
    kegg_rxn_field = record.get("KEGG_Reaction", "").strip()
    ms_rxn_entries = []
    if kegg_rxn_field and kegg_rxn_field != "-":
        # There might be multiple KEGG reactions separated by commas.
        kegg_rxns = [rxn.strip() for rxn in kegg_rxn_field.split(",")]
        for kegg_rxn in kegg_rxns:
            entries = kegg_mapping.get(kegg_rxn.upper(), [])
            for entry in entries:
                ms_rxn_entries.append(entry)
    record["modelseed_reactions"] = ms_rxn_entries

# 7. Parse transporter annotations and add transportable metabolites info.
transporter_file = "TCDB_transporter_substrates.txt"
transporter_dict = parse_transporter(transporter_file)
for query_id, record in eggnog_data.items():
    kegg_tc_field = record.get("KEGG_TC", "").strip()
    metabolite_entries = []
    if kegg_tc_field and kegg_tc_field != "-":
        tc_ids = [tc.strip() for tc in kegg_tc_field.split(",")]
        for tc in tc_ids:
            # For partial/incomplete codes, report any transporter code that starts with the eggnog TC.
            for tc_code, subs in transporter_dict.items():
                if tc_code.startswith(tc):
                    metabolite_entries.extend(subs)
    record["transportable_metabolites"] = metabolite_entries

Parsed eggnog file: 3839 records found.
Parsed FASTA file: 3983 protein sequences found.
Parsed GFF file: 8168 gene locus entries found.
Downloaded 33992 compounds.
Cleaning and deduplicating ModelSEED reactions...
Downloaded 43774 reactions.
Reactions with status 'OK': 32014 (out of 43774 downloaded)
Built cleaned KEGG-to-ModelSEED mapping: 7673 KEGG reaction entries mapped.


In [5]:
#%% Examples of retrieving specific records from eggnog_data
if eggnog_data:
    sample_query = list(eggnog_data.keys())[333]
    print("\n--- Example Record ---")
    print(f"Query ID: {sample_query}")
    print("Record details:")
    print(eggnog_data[sample_query])

    protein_seq = eggnog_data[sample_query].get("protein_seq", "")
    locus = eggnog_data[sample_query].get("locus", "")
    print("\nProtein sequence (first 60 aa):")
    print(protein_seq[:60] + "..." if protein_seq else "Not found")
    print("\nLocus (start-end):")
    print(locus)

    print("\nModelSEED reaction entries:")
    print(eggnog_data[sample_query].get("modelseed_reactions", []))
    print("\nTransportable metabolites:")
    print(eggnog_data[sample_query].get("transportable_metabolites", []))
else:
    print("No eggnog records found.")


--- Example Record ---
Query ID: WP_014948679.1
Record details:
{'query': 'WP_014948679.1', 'seed_ortholog': '1004785.AMBLS11_05215', 'evalue': '1.03e-218', 'score': '605.0', 'eggNOG_OGs': 'COG0601@1|root,COG0601@2|Bacteria,1MU8Z@1224|Proteobacteria,1RNJ1@1236|Gammaproteobacteria,464TR@72275|Alteromonadaceae', 'max_annot_lvl': '1236|Gammaproteobacteria', 'COG_category': 'P', 'Description': 'COG0601 ABC-type dipeptide oligopeptide nickel transport systems, permease components', 'Preferred_name': 'dppB', 'GOs': 'GO:0005575,GO:0005623,GO:0005886,GO:0016020,GO:0044464,GO:0071944', 'EC': '-', 'KEGG_ko': 'ko:K12369,ko:K15581,ko:K19227', 'KEGG_Pathway': 'ko01501,ko01503,ko02010,ko02024,map01501,map01503,map02010,map02024', 'KEGG_Module': 'M00324,M00439,M00739', 'KEGG_Reaction': '-', 'KEGG_rclass': '-', 'BRITE': 'ko00000,ko00001,ko00002,ko02000', 'KEGG_TC': '3.A.1.5,3.A.1.5.1,3.A.1.5.18,3.A.1.5.19,3.A.1.5.25,3.A.1.5.5', 'CAZy': '-', 'BiGG_Reaction': 'iECUMN_1333.ECUMN_4053', 'PFAMs': 'BPD_tra

In [ ]:
gff_dict

## Function to Export EggNog structure to Excel

In [6]:
# STANDALONE FUNCTION TO EXPORT EggNOG structure TO EXCEL
# Can be later substituted to equivalent part above
def export_eggnog_to_excel(eggnog_data, output_filename=None):
    """
    Exports the eggnog_data structure to an Excel file.

    The function creates separate columns for:
      - ModelSeed_Reaction_IDs and ModelSeed_Reaction_Names
      - Transportable_Metabolite_CHEBI and Transportable_Metabolite_Names

    If output_filename is not provided, the file is named as "eggnog_data_<timestamp>.xlsx".
    """
    flat_records = []
    for query_id, record in eggnog_data.items():
        flat_record = {}
        # Copy over non-complex fields.
        for key, value in record.items():
            if key in ["modelseed_reactions", "transportable_metabolites"]:
                continue
            if isinstance(value, (dict, list)):
                flat_record[key] = json.dumps(value)
            else:
                flat_record[key] = value

        # Process ModelSEED reactions separately.
        ms_reactions = record.get("modelseed_reactions", [])
        ms_ids = []
        ms_names = []
        for entry in ms_reactions:
            ms_ids.append(entry.get("id", ""))
            ms_names.append(entry.get("name", ""))
        flat_record["Manual_ModelSeed_rxn_ID"] = ",".join(ms_ids)
        flat_record["Manual_ModelSeed_rxn_Name"] = ",".join(ms_names)

        # Process transportable metabolites separately.
        t_metabolites = record.get("transportable_metabolites", [])
        t_chebi = []
        t_names = []
        for entry in t_metabolites:
            t_chebi.append(entry.get("chebi", ""))
            t_names.append(entry.get("name", ""))
        flat_record["Transportable_Metabolite_CHEBI"] = ",".join(t_chebi)
        flat_record["Transportable_Metabolite_Names"] = ",".join(t_names)

        flat_records.append(flat_record)

    df = pd.DataFrame(flat_records)
    if not output_filename:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_filename = f"eggnog_data_{timestamp}.xlsx"
    df.to_excel(output_filename, index=False)
    print(f"\nExcel file exported as: {output_filename}")

# ------------------------------------------------------------------
# USAGE EXAMPLE:
# Assume eggnog_data is already in memory.
# Call the function as follows:
#
# export_eggnog_to_excel(eggnog_data)
#
# Or, if you prefer to specify a filename:
#
# export_eggnog_to_excel(eggnog_data, output_filename="my_eggnog_data.xlsx")


**Export Excel file with all records**

In [34]:
export_eggnog_to_excel(eggnog_data)


Excel file exported as: eggnog_data_20250227_174641.xlsx


## Parse SBML (XML) Model file from KBase to extract reactions associated with each gene, and add these reactions to corresponding entry in EggNOG data structure

Require presence of xml file (Currently 'mit1002_rast_omegga.mdl.xml')

In [35]:
# UNIVERSAL MODEL UPLOAD FUNCTION
# This function detects automatically the format of a model and uploads it using cobrapy
# Possible formats: .mat, json, .xml
def read_model(_file_name):
  suffix = _file_name.split('.')[-1]
  print("Uploading model of type: ",suffix)
  if (suffix == 'xml'):
    _model = cobra.io.read_sbml_model(_file_name)
  elif (suffix == 'mat'):
    _model = cobra.io.load_matlab_model(_file_name)
  elif (suffix == 'json'):
    _model = cobra.io.load_json_model(_file_name)
  else:
    print("MODEL OF UNKNOWN TYPE")
    _model='UNKNOWN'
  n = len(_model.reactions)
  m = len(_model.metabolites)
  print("Number of reactions = ",len(_model.reactions))
  print("Number of metabolites = ",len(_model.metabolites))
  print("Number of genes = ",len(_model.genes))
  return _model


In [36]:
import cobra

filename = 'hot1a3_genome.rast.mdl.xml'

# Load the model using COBRApy's SBML reader.
# If your model is in a different supported format, use the appropriate reader.
#model = cobra.io.read_sbml_model(filename)
model = read_model(filename)
# Print a summary of the loaded model
print("Model loaded successfully!")
print("Model name:", model.name)
print("Number of reactions:", len(model.reactions))
print("Number of metabolites:", len(model.metabolites))
print("Number of genes:", len(model.genes))


ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.


Uploading model of type:  xml
Number of reactions =  1342
Number of metabolites =  1208
Number of genes =  916
Model loaded successfully!
Model name: None
Number of reactions: 1342
Number of metabolites: 1208
Number of genes: 916


In [41]:
def map_gene_to_protein(filename):
    """
    Parses a GFF file and returns a dictionary mapping gene names (extracted from the 'ID' attribute of gene features)
    to their corresponding protein names (extracted from the 'ID' attribute of CDS features).
    """
    gene_to_protein = {}

    # Read GFF file using gffpandas
    gff_df = gffpd.read_gff3(filename).df

    # Extract 'gene' and 'CDS' features
    genes_df = gff_df[gff_df["type"] == "gene"]
    cds_df = gff_df[gff_df["type"] == "CDS"]

    # Create a mapping of gene ID to locus_tag (e.g., ACZ81_RS00010)
    gene_map = {}
    for _, row in genes_df.iterrows():
        attributes = row["attributes"]
        attr_dict = dict(item.split("=") for item in attributes.split(";") if "=" in item)

        if "ID" in attr_dict:
            gene_id = attr_dict["ID"].replace("gene-", "")  # Remove "gene-" prefix
            if "locus_tag" in attr_dict:
                locus_tag = attr_dict["locus_tag"]
                gene_map[locus_tag] = gene_id

    # Create a mapping of locus_tag to protein ID (from CDS features)
    for _, row in cds_df.iterrows():
        attributes = row["attributes"]
        attr_dict = dict(item.split("=") for item in attributes.split(";") if "=" in item)

        if "ID" in attr_dict and "locus_tag" in attr_dict:
            protein_id = attr_dict["ID"].replace("cds-", "")  # Remove "cds-" prefix
            locus_tag = attr_dict["locus_tag"]

            if locus_tag in gene_map:
                gene_name = gene_map[locus_tag]
                gene_to_protein[gene_name] = protein_id

    return gene_to_protein



In [42]:
# Example usage:
gene_protein_map = map_gene_to_protein("genomic.gff")
print(gene_protein_map)


{'ACZ81_RS00010': 'WP_012516526.1', 'ACZ81_RS00015': 'WP_061438857.1', 'ACZ81_RS00020': 'WP_014977848.1', 'ACZ81_RS00025': 'WP_039234509.1', 'ACZ81_RS00030': 'WP_061438859.1', 'ACZ81_RS00035': 'WP_014947716.1', 'ACZ81_RS00040': 'WP_080986385.1', 'ACZ81_RS00045': 'WP_049586362.1', 'ACZ81_RS00050': 'WP_014947719.1', 'ACZ81_RS00055': 'WP_080986387.1', 'ACZ81_RS00060': 'WP_014947721.1', 'ACZ81_RS00065': 'WP_014975249.1', 'ACZ81_RS00070': 'WP_061438861.1', 'ACZ81_RS00075': 'WP_061438863.1', 'ACZ81_RS00080': 'WP_014947725.1', 'ACZ81_RS00085': 'WP_014947726.1', 'ACZ81_RS00090': 'WP_061438869.1', 'ACZ81_RS00095': 'WP_061438871.1', 'ACZ81_RS00100': 'WP_014947729.1', 'ACZ81_RS00105': 'WP_061486884.1', 'ACZ81_RS00110': 'WP_014975255.1', 'ACZ81_RS00115': 'WP_014947732.1', 'ACZ81_RS00120': 'WP_014975256.1', 'ACZ81_RS00125': 'WP_014975257.1', 'ACZ81_RS00130': 'WP_049586350.1', 'ACZ81_RS00135': 'WP_061438872.1', 'ACZ81_RS00140': 'WP_014947737.1', 'ACZ81_RS00145': 'WP_061438874.1', 'ACZ81_RS00150': 'W

In [43]:
# Build a gene-to-reaction mapping by iterating over model.genes.
# For each gene, list all reaction IDs where it appears.
gene_mapping = []
for gene in model.genes:
    protein_to_check = gene_protein_map.get(gene.id)
    if not protein_to_check:
        print(f"No protein found for gene {gene.id}. Skipping.")
        continue
    reaction_ids = [rxn.id for rxn in gene.reactions]
    # Remove the '_gene' suffix if present.
    # gene_id = gene.id
    # if gene_id.endswith("_gene"):
    #     gene_id = gene_id.replace("_gene", "")
    gene_mapping.append({
        "gene": protein_to_check,
        "reactions": reaction_ids
    })

# Convert the mapping into a DataFrame and display it
gene_df = pd.DataFrame(gene_mapping)
#print("Gene-to-Reaction Mapping:")
#display(gene_df)


No protein found for gene ACZ81_RS20210. Skipping.
No protein found for gene ACZ81_RS20710. Skipping.
No protein found for gene ACZ81_RS17610. Skipping.
No protein found for gene ACZ81_RS20695. Skipping.
No protein found for gene ACZ81_RS18560. Skipping.


In [44]:
gene_df

,gene,reactions
0,WP_061485809.1,"[rxn02201_c0, rxn02200_c0]"
1,WP_014948272.1,"[rxn02201_c0, rxn02200_c0, rxn02503_c0]"
2,WP_014951000.1,"[rxn02201_c0, rxn02200_c0, rxn02503_c0]"
3,WP_061439009.1,[rxn00351_c0]
4,WP_014950459.1,[rxn00351_c0]
...,...,...
906,WP_232375997.1,[rxn19704_c0]
907,WP_061487036.1,[rxn19704_c0]
908,WP_061439380.1,[rxn43657_c0]
909,WP_061486181.1,[rxn45692_c0]


In [45]:
# Create a dictionary mapping from gene IDs (from gene_df) to their reaction lists
gene_dict = dict(zip(gene_df["gene"], gene_df["reactions"]))

# Iterate over each record in eggnog_data (keyed by the "query" field)
for query_id, record in eggnog_data.items():
    # Look up the reaction list for the gene (query_id) and add it to the record
    record["KBase_rxns_draft"] = gene_dict.get(query_id, [])

# Optional: Display the first few records to verify the new field has been added
# for i, (gene, record) in enumerate(eggnog_data.items()):
#     print(f"Gene: {gene}")
#     print(record)
#     print("-" * 40)
#     if i >= 4:  # show first 5 records
#         break


In [46]:
export_eggnog_to_excel(eggnog_data)


Excel file exported as: eggnog_data_20250227_175723.xlsx


# Generate tables of reactions and list of transportable metabolites

In [ ]:
# import pandas as pd
# from datetime import datetime

# # === 1. Collect all unique ModelSEED reactions present in the genome ===
# ms_reaction_dict = {}
# for gene_id, record in eggnog_data.items():
#     for rxn in record.get("modelseed_reactions", []):
#         rxn_id = rxn.get("id", "").strip()
#         if rxn_id:
#             # Use the reaction id as key; if already present, we assume names are the same.
#             ms_reaction_dict[rxn_id] = rxn.get("name", "").strip()

# # Convert to a list of dictionaries for DataFrame creation.
# ms_reactions_list = [
#     {"Reaction_ID": rid, "Reaction_Name": name}
#     for rid, name in ms_reaction_dict.items()
# ]

# # === 2. Collect all unique transportable metabolites ===
# # Each metabolite is defined by its CHEBI, common name, ModelSEED compound ID, and ModelSEED compound name.
# tm_dict = {}
# for gene_id, record in eggnog_data.items():
#     for tm in record.get("transportable_metabolites", []):
#         chebi = tm.get("chebi", "").strip()
#         common_name = tm.get("name", "").strip()
#         ms_comp = tm.get("modelseed", "").strip()
#         ms_comp_name = tm.get("modelseed_name", "").strip()
#         # Use a tuple key to guarantee uniqueness.
#         key = (chebi, common_name, ms_comp, ms_comp_name)
#         tm_dict[key] = {
#             "CHEBI": chebi,
#             "Common_Name": common_name,
#             "ModelSeed": ms_comp,
#             "ModelSeed_Name": ms_comp_name
#         }

# tm_list = list(tm_dict.values())

# # === 3. Create DataFrames for reactions and metabolites ===
# df_reactions = pd.DataFrame(ms_reactions_list)
# df_metabolites = pd.DataFrame(tm_list)

# # === 4. Write both DataFrames to a single Excel file with two separate sheets ===
# timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
# excel_filename = f"genome_reactions_and_metabolites_{timestamp}.xlsx"

# with pd.ExcelWriter(excel_filename) as writer:
#     df_reactions.to_excel(writer, sheet_name="ModelSeed Reactions", index=False)
#     df_metabolites.to_excel(writer, sheet_name="Transportable Metabolites", index=False)

# print(f"Excel file exported as: {excel_filename}")


Excel file exported as: genome_reactions_and_metabolites_20250214_033007.xlsx


# Add CarveMe output info (BiGG name, score) to egg_nog data frame

In [57]:
import re

def parse_carveme_model_scores(carveme_file, eggnog_data):
    """
    Parses carveme_model_reaction_scores.tsv and adds two new fields to eggnog_data:
      - CarveME_BiGG_R: list of reaction IDs (e.g., R_3HAD80, R_ACOAD2, etc.)
      - CarveMe_score: list of corresponding normalized scores.

    Matching is based on protein IDs extracted from the GPR column (e.g., WP_014949559.1),
    which are compared to the eggnog_data keys (query IDs). If a query_id is not yet in
    eggnog_data, a new record is created for it.
    """
    with open(carveme_file, 'r') as f:
        # Read the header
        header = next(f).strip().split('\t')
        # Expecting columns: reaction, GPR, score, normalized_score
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split('\t')
            if len(parts) < 4:
                continue

            reaction, gpr, score_str, norm_score_str = parts

            # Use regex to extract all protein IDs from the GPR column.
            # This pattern assumes protein IDs look like WP_<digits>.<digits>
            matches = re.findall(r'WP_\d+\.\d+', gpr)
            # Uncomment the following line to debug the extraction:
            # print("GPR:", gpr, "->", matches)

            for query_id in matches:
                # If this query_id is not in eggnog_data, create a new record
                if query_id not in eggnog_data:
                    eggnog_data[query_id] = {}

                record = eggnog_data[query_id]

                # Initialize the fields if they don't exist
                if "CarveME_BiGG_R" not in record:
                    record["CarveME_BiGG_R"] = []
                if "CarveMe_score" not in record:
                    record["CarveMe_score"] = []

                # Append the reaction and normalized score
                record["CarveME_BiGG_R"].append(reaction)
                record["CarveMe_score"].append(norm_score_str)


In [54]:
# def parse_carveme_model_scores(carveme_file, eggnog_data):
#     """
#     Parses carveme_model_reaction_scores.tsv and adds two new fields to eggnog_data:
#       - CarveME_BiGG_R: list of reaction IDs (e.g., R_3HAD80, R_ACOAD2, etc.)
#       - CarveMe_score: list of corresponding normalized scores.

#     Matching is based on the number following "anvio_prot_seqs_" in the GPR column,
#     which is compared to the eggnog_data keys (query IDs). If a query_id is not yet in
#     eggnog_data, a new record is created for it.
#     """
#     with open(carveme_file, 'r') as f:
#         # Read the header
#         header = next(f).strip().split('\t')
#         # Expecting columns: reaction, GPR, score, normalized_score
#         for line in f:
#             line = line.strip()
#             if not line:
#                 continue
#             parts = line.split('\t')
#             if len(parts) < 4:
#                 continue

#             reaction, gpr, score_str, norm_score_str = parts

#             # Find all occurrences of anvio_prot_seqs_XXXX in the GPR column
#             matches = [gpr]
#             print(gpr)

#             for query_id in matches:
#                 # If this query_id is not in eggnog_data, create a new record
#                 if query_id not in eggnog_data:
#                     eggnog_data[query_id] = {}

#                 record = eggnog_data[query_id]

#                 # Initialize the fields if they don't exist
#                 if "CarveME_BiGG_R" not in record:
#                     record["CarveME_BiGG_R"] = []
#                 if "CarveMe_score" not in record:
#                     record["CarveMe_score"] = []

#                 # Append the reaction and normalized score
#                 record["CarveME_BiGG_R"].append(reaction)
#                 record["CarveMe_score"].append(norm_score_str)


In [58]:
carveme_file = "carveme_model_reaction_scores.tsv"
parse_carveme_model_scores(carveme_file, eggnog_data)

## Finally export everything to Excel

In [59]:
export_eggnog_to_excel(eggnog_data)


Excel file exported as: eggnog_data_20250227_180812.xlsx


## Extra

In [ ]:
import re

def report_carveme_statistics(carveme_file, eggnog_data):
    """
    Reads carveme_model_reaction_scores.tsv and reports statistics:
      - Total number of lines (reactions) in the carveme file.
      - Total distinct CarveMe reaction IDs found in the file.
      - Number of those reactions that had at least one match in eggnog_data.
      - Total number of eggnog queries.
      - Number of eggnog queries that received CarveMe annotations.
      - Total number of CarveMe reaction entries added to eggnog_data.
    """
    total_carveme_reactions = set()     # distinct reaction IDs in the carveme file
    matched_carveme_reactions = set()   # reactions that had a match in eggnog_data
    total_lines = 0

    with open(carveme_file, 'r') as f:
        header = next(f)  # Skip header
        for line in f:
            total_lines += 1
            line = line.strip()
            if not line:
                continue
            parts = line.split('\t')
            if len(parts) < 4:
                continue
            reaction, gpr, score_str, norm_score_str = parts
            total_carveme_reactions.add(reaction)

            # Find all occurrences of anvio_prot_seqs_XXXX in the GPR column
            query_ids = re.findall(r'anvio_prot_seqs_(\d+)', gpr)
            # If any of these query IDs is in eggnog_data, mark this reaction as matched.
            for qid in query_ids:
                if qid in eggnog_data:
                    matched_carveme_reactions.add(reaction)
                    break  # no need to check further for this reaction

    # Now scan eggnog_data to see how many queries have CarveMe matches
    queries_with_carveme = 0
    total_carveme_matches = 0
    for query, record in eggnog_data.items():
        if "CarveME_BiGG_R" in record:
            queries_with_carveme += 1
            total_carveme_matches += len(record["CarveME_BiGG_R"])

    print("=== CarveMe File Statistics ===")
    print("Total lines in carveme file:", total_lines)
    print("Total distinct CarveMe reaction IDs in file:", len(total_carveme_reactions))
    print("CarveMe reactions with a match in eggnog:", len(matched_carveme_reactions))
    print("Total eggnog queries:", len(eggnog_data))
    print("Eggnog queries with CarveMe matches:", queries_with_carveme)
    print("Total number of CarveMe reaction entries in eggnog_data:", total_carveme_matches)

# ------------------------------------------------------------------
# USAGE EXAMPLE:
# Assuming that:
#   - Your eggnog_data dictionary is already populated.
#   - You have called parse_carveme_model_scores(carveme_file, eggnog_data)
#
# Then you can call:
#
# carveme_file = "carveme_model_reaction_scores.tsv"
# report_carveme_statistics(carveme_file, eggnog_data)
#
# This will print out statistics that help you understand how many reactions
# in the CarveMe file had matches in eggnog_data and vice versa.


In [ ]:
carveme_file = "carveme_model_reaction_scores.tsv"
report_carveme_statistics(carveme_file, eggnog_data)

=== CarveMe File Statistics ===
Total lines in carveme file: 399
Total distinct CarveMe reaction IDs in file: 399
CarveMe reactions with a match in eggnog: 328
Total eggnog queries: 3929
Eggnog queries with CarveMe matches: 183
Total number of CarveMe reaction entries in eggnog_data: 393
